In [61]:
%%time
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
import time


raw_path = 'C:\\Finance_team\\raw_data' 
out_path = 'C:\\Finance_team\\Testing_result'
file_list = os.listdir('C:\\Finance_team\\raw_data')

product_id = ''
product_code = ''
date = ''
all_columns = ['시간', '현재가', '체결량', '투자자', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']
int_columns = ['현재가', '체결량', '투자자정보', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']

def is_target_product_ID(line):
    return line[17:19] == 'A0' and line[425:428] == 'BM3' and line[471:474] == '001'

for i in file_list:
    print(i)
    with open('{0}\\{1}'.format(raw_path, i), 'r', encoding='ISO-8859-1') as f:
        result = []
        lines = f.readlines()
        data_in_array = []

        for line in lines:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                break
        
        for line in lines:
            data_in_array.append(np.array(line.split(',')))
        
        for line in data_in_array:
            
            if (line[0][1:3] < '09'): continue

            if (line[0][1:3] == '15' and line[0][4:6] == '45' and line[0][7:9] >= '01'): break
                  
            if (line[0][22:34] == product_code) or (line[0][38:49] == product_id):
        
                if line[0][17:19] == 'A3':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],np.nan, 
                                    np.nan, np.nan, np.nan, np.nan, line[0][152:153]])
                elif line[0][17:19] == 'B6':
                    result.append([line[0][:17], np.nan, np.nan, np.nan, line[0][57:63], 
                                    line[0][139:145], np.nan, np.nan, np.nan])  
                elif line[0][17:19] == 'G7':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],
                                   np.nan, line[0][169:175], line[0][251:257], 
                                   np.nan, np.nan, line[0][152:153]])
                elif (line[0][17:19] == 'H1' and (line[0][49:53] == '1000')):
                    if (line[0][1:3] == '15' and line[0][4:6] == '35' and line[0][7:9] >= '01'): break
                    result.append([line[0][:17], np.nan, np.nan, line[0][49:53],
                                  np.nan, np.nan, line[0][53:62], line[0][62:71],
                                  np.nan])
            
        result_frame = pd.DataFrame(result, columns = all_columns)
        result_frame.drop(0, inplace=True)
        
result_frame

sise2local.ALL_CMDT.0201
Wall time: 6.05 s


In [62]:
result_frame.head()

,시간,현재가,체결량,투자자,매수1호가수량,매도1호가수량,투자자매수량,투자자매도량,매도or매수
1,[09:00:01-039271],NaN,NaN,NaN,000850,000187,NaN,NaN,NaN
2,[09:00:01-066900],NaN,NaN,NaN,000850,000187,NaN,NaN,NaN
3,[09:00:01-090882],00010745,000001,NaN,000849,000187,NaN,NaN,1
4,[09:00:01-103833],NaN,NaN,NaN,000852,000187,NaN,NaN,NaN
5,[09:00:01-105408],NaN,NaN,NaN,000003,000187,NaN,NaN,NaN


#  H1_Flag Data Frame 생성 

In [63]:
h1 = result_frame.copy()
h1 = h1[(h1['투자자'] == '1000')]
h1.iloc[:, 1:] = h1.iloc[:, 1:].astype('float')

h1 = h1[['시간', '투자자매수량', '투자자매도량']]
# h1['투자자매수량변화(%)'] = round(h1['투자자매수량'].pct_change() * 100, 1)
# h1['투자자매도량변화(%)'] = round(h1['투자자매도량'].pct_change() * 100, 2)
h1['투자자매수량변화'] = h1['투자자매수량'].diff()
h1['투자자매도량변화'] = h1['투자자매도량'].diff()
h1['투자자변화량차이'] = h1['투자자매수량변화'] - h1['투자자매도량변화']

h1['H1_Flag'] = np.nan
h1['H1_Flag'][h1['투자자변화량차이'] > 0] = 1
h1['H1_Flag'][h1['투자자변화량차이'] < 0] = -1
h1['H1_Flag'][h1['투자자변화량차이'] == 0] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [64]:
h1.head()

,시간,투자자매수량,투자자매도량,투자자매수량변화,투자자매도량변화,투자자변화량차이,H1_Flag
166,[09:00:36-918856],2734,1239,NaN,NaN,NaN,NaN
227,[09:01:04-218414],3014,1559,280,320,-40,-1.0
267,[09:01:34-112279],3046,1591,32,32,0,0.0
397,[09:02:04-268661],3604,2049,558,458,100,1.0
506,[09:02:34-022089],4294,2612,690,563,127,1.0


# Rest_Flag Data Frame 만들기

In [65]:
# 먼저 Current Price정리 
rest = result_frame.copy()
rest = rest[rest['투자자'] != '1000']
rest = rest[['시간', '현재가', '체결량', '매수1호가수량', '매도1호가수량', '매도or매수']]
rest.iloc[:, 1:] = rest.iloc[:, 1:].astype('float')

# rest['현재가'].fillna(method='ffill', inplace=True)
rest['매수1호가수량-매도1호가수량'] = rest['매수1호가수량'] - rest['매도1호가수량']
rest['Rest_Flag'] = np.nan
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] > 0] = 1
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] == 0] = 0
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] < 0] = -1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


# (H1_Flag `+` Rest_Flag) Data Frame 만들기 

In [69]:
df2 = pd.concat([rest, h1])
df2.sort_index(inplace=True)

cols = ['시간', '현재가', '체결량', '매도or매수', '매수1호가수량', '매도1호가수량', 
       '매수1호가수량-매도1호가수량', 'Rest_Flag', '투자자매수량', '투자자매도량',
       '투자자매수량변화', '투자자매도량변화', '투자자변화량차이', 'H1_Flag']

df_test = df2[cols]

df_test['현재가'].fillna(method='ffill', inplace=True)
df_test['H1_Flag'].fillna(method='ffill', inplace=True)
df_test['Rest_Flag'].fillna(method='ffill', inplace=True)
df_test.loc[1550:1560]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,시간,현재가,체결량,매도or매수,매수1호가수량,매도1호가수량,매수1호가수량-매도1호가수량,Rest_Flag,투자자매수량,투자자매도량,투자자매수량변화,투자자매도량변화,투자자변화량차이,H1_Flag
1550,[09:12:33-227538],10743.0,NaN,NaN,356,434,-78,-1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1551,[09:12:33-228794],10743.0,NaN,NaN,356,431,-75,-1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1552,[09:12:33-230058],10743.0,NaN,NaN,356,428,-72,-1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1553,[09:12:33-366340],10743.0,NaN,NaN,381,428,-47,-1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1554,[09:12:33-408629],10743.0,NaN,NaN,481,428,53,1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1555,[09:12:34-017151],10743.0,NaN,NaN,561,428,133,1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1556,[09:12:34-125193],10743.0,NaN,NaN,NaN,NaN,NaN,1.0,8838,5366,129,390,-261,-1.0
1557,[09:12:34-152294],10744.0,1,2,561,427,134,1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1558,[09:12:34-939920],10744.0,NaN,NaN,571,427,144,1.0,NaN,NaN,NaN,NaN,NaN,-1.0
1559,[09:12:35-857846],10744.0,NaN,NaN,571,427,144,1.0,NaN,NaN,NaN,NaN,NaN,-1.0


# 전략 검증 위해 Test Frame 생성 as 'a'

In [70]:
a = df_test[['시간', '현재가', '매수1호가수량', '매도1호가수량', 'H1_Flag', 'Rest_Flag']]

In [71]:
a.loc[350:400]

,시간,현재가,매수1호가수량,매도1호가수량,H1_Flag,Rest_Flag
350,[09:01:55-793040],10746.0,159,845,0.0,-1.0
351,[09:01:55-799100],10746.0,156,845,0.0,-1.0
352,[09:01:55-800526],10746.0,156,870,0.0,-1.0
353,[09:01:55-803099],10746.0,1248,870,0.0,1.0
354,[09:01:55-806673],10746.0,1223,870,0.0,1.0
355,[09:01:56-252896],10746.0,1223,880,0.0,1.0
356,[09:01:56-433787],10746.0,1232,880,0.0,1.0
357,[09:01:56-434856],10746.0,1246,880,0.0,1.0
358,[09:01:56-695485],10746.0,1232,880,0.0,1.0
359,[09:01:56-696706],10746.0,1223,880,0.0,1.0


# H1_Flag 및 Rest_Flag 의 0 값을 np.nan값으로 변환

In [72]:
a['H1_Flag'][a['H1_Flag'] == 0] = np.nan
a['Rest_Flag'][a['Rest_Flag'] == 0] = np.nan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C

# H1_Flag 및 Rest_Flag의 np.nan 값을 fillna의 method='ffill'을 활용해서 이전 값 참조

In [73]:
a['H1_Flag'].fillna(method='ffill', inplace=True)
a['Rest_Flag'].fillna(method='ffill', inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [74]:
a.loc[:,'long_signal'] = np.nan
a.loc[:,'long_close_signal'] = np.nan
a

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,시간,현재가,매수1호가수량,매도1호가수량,H1_Flag,Rest_Flag,long_signal,long_close_signal
1,[09:00:01-039271],NaN,850,187,NaN,1.0,NaN,NaN
2,[09:00:01-066900],NaN,850,187,NaN,1.0,NaN,NaN
3,[09:00:01-090882],10745.0,849,187,NaN,1.0,NaN,NaN
4,[09:00:01-103833],10745.0,852,187,NaN,1.0,NaN,NaN
5,[09:00:01-105408],10745.0,3,187,NaN,-1.0,NaN,NaN
6,[09:00:01-168950],10745.0,852,187,NaN,1.0,NaN,NaN
7,[09:00:01-198520],10745.0,852,240,NaN,1.0,NaN,NaN
8,[09:00:01-501859],10745.0,852,240,NaN,1.0,NaN,NaN
9,[09:00:01-931889],10745.0,552,240,NaN,1.0,NaN,NaN
10,[09:00:02-120073],10745.0,552,540,NaN,1.0,NaN,NaN


In [32]:
# renew_index_long = 0
# # print(renew_index, '#0')

# while True:
#     for index, row in a.loc[renew_index_long:].iterrows():

#         if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1: 
#             renew_index_long = index
#             print(row['H1_Flag'], row['Rest_Flag'])
#             print(renew_index_long,'#1 signal -1 , 1')
            
#             for index, row in a.loc[renew_index_long:].iterrows():

#                 if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
#                     renew_index_long = index
#                     print(row['H1_Flag'], row['Rest_Flag'])
#                     print(renew_index_long, '# 2 signal -1 , -1')
#                     break
#             print('########################')
#             break
#     if index == len(a.loc[:]):
#         break

        
# # 여기서, index와 row 를 활용해서 시그널을 제대로 찾아야 함.
# # 집중해라 여기만 통과하면 끝이다.

# # omg...

-1.0 1.0
227 #1 signal -1 , 1
-1.0 -1.0
231 # 2 signal -1 , -1
########################
-1.0 1.0
280 #1 signal -1 , 1
-1.0 -1.0
344 # 2 signal -1 , -1
########################
-1.0 1.0
353 #1 signal -1 , 1
-1.0 -1.0
375 # 2 signal -1 , -1
########################
-1.0 1.0
543 #1 signal -1 , 1
-1.0 -1.0
551 # 2 signal -1 , -1
########################
-1.0 1.0
553 #1 signal -1 , 1
-1.0 -1.0
590 # 2 signal -1 , -1
########################
-1.0 1.0
1115 #1 signal -1 , 1
-1.0 -1.0
1135 # 2 signal -1 , -1
########################
-1.0 1.0
1144 #1 signal -1 , 1
-1.0 -1.0
1161 # 2 signal -1 , -1
########################
-1.0 1.0
1171 #1 signal -1 , 1
-1.0 -1.0
1193 # 2 signal -1 , -1
########################
-1.0 1.0
1203 #1 signal -1 , 1
-1.0 -1.0
1206 # 2 signal -1 , -1
########################
-1.0 1.0
1207 #1 signal -1 , 1
-1.0 -1.0
1218 # 2 signal -1 , -1
########################
-1.0 1.0
1338 #1 signal -1 , 1
-1.0 -1.0
1357 # 2 signal -1 , -1
########################
-1.0 1.0
1455 #1 sig

-1.0 1.0
10267 #1 signal -1 , 1
-1.0 -1.0
10300 # 2 signal -1 , -1
########################
-1.0 1.0
11313 #1 signal -1 , 1
-1.0 -1.0
11315 # 2 signal -1 , -1
########################
-1.0 1.0
11325 #1 signal -1 , 1
-1.0 -1.0
11638 # 2 signal -1 , -1
########################
-1.0 1.0
11746 #1 signal -1 , 1
-1.0 -1.0
11763 # 2 signal -1 , -1
########################
-1.0 1.0
12124 #1 signal -1 , 1
-1.0 -1.0
12132 # 2 signal -1 , -1
########################
-1.0 1.0
12135 #1 signal -1 , 1
-1.0 -1.0
12137 # 2 signal -1 , -1
########################
-1.0 1.0
12142 #1 signal -1 , 1
-1.0 -1.0
12796 # 2 signal -1 , -1
########################
-1.0 1.0
12801 #1 signal -1 , 1
-1.0 -1.0
12804 # 2 signal -1 , -1
########################
-1.0 1.0
12807 #1 signal -1 , 1
-1.0 -1.0
13127 # 2 signal -1 , -1
########################
-1.0 1.0
13574 #1 signal -1 , 1
-1.0 -1.0
13578 # 2 signal -1 , -1
########################
-1.0 1.0
13583 #1 signal -1 , 1
-1.0 -1.0
13678 # 2 signal -1 , -1
#############

In [39]:
# renew_index_long = 0
# holding_position = 0
# # print(renew_index, '#0')

# while True:
#     for index, row in a.loc[renew_index_long:].iterrows():
    
#         if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1: 
#             renew_index_long = index
            
#             print(row['H1_Flag'], row['Rest_Flag'])
#             print(renew_index_long,'#1 long signal -1 , +1')
            
#             for index, row in a.loc[renew_index_long:].iterrows():

#                 if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
#                     renew_index_long = index
                    
#                     holding_position += 1
                    
#                     print(row['H1_Flag'], row['Rest_Flag'])
#                     print(renew_index_long, '#2 long signal -1 , -1')
                    
#                     print('##########################################')
                    
#                     for index, row in a.loc[renew_index_long:].iterrows():
                        
#                         if row['H1_Flag'] == 1 and row['Rest_Flag'] -1:
#                             renew_index_long = index
#                             print(row['H1_Flag'], row['Rest_Flag'])
#                             print(renew_index_long, '#1 close signal +1, -1')
                            
#                             for index, row in a.loc[renew_index_long:].iterrows():
                                
#                                 if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1:
#                                     renew_index_long = index
#                                     print(row['H1_Flag'] , row['Rest_Flag'])
#                                     print(renew_index_long, '#2 close signal +1, +1')
                                                       
#                                     print('########################################')
                                                       
#                                     break
#                             break
#                     break
#             break                    
        
        
#     if index == len(a.loc[:]):
#         break


-1.0 1.0
227 #1 long signal -1 , +1
-1.0 -1.0
231 #2 long signal -1 , -1
##########################################
1.0 -1.0
397 #1 close signal +1, -1
1.0 1.0
408 #2 close signal +1, +1
########################################
-1.0 1.0
543 #1 long signal -1 , +1
-1.0 -1.0
551 #2 long signal -1 , -1
##########################################
1.0 -1.0
642 #1 close signal +1, -1
1.0 1.0
678 #2 close signal +1, +1
########################################
-1.0 1.0
1115 #1 long signal -1 , +1
-1.0 -1.0
1135 #2 long signal -1 , -1
##########################################
1.0 -1.0
1234 #1 close signal +1, -1
1.0 1.0
1241 #2 close signal +1, +1
########################################
-1.0 1.0
1338 #1 long signal -1 , +1
-1.0 -1.0
1357 #2 long signal -1 , -1
##########################################
1.0 -1.0
1402 #1 close signal +1, -1
1.0 1.0
1600 #2 close signal +1, +1
########################################
-1.0 1.0
1688 #1 long signal -1 , +1
-1.0 -1.0
2002 #2 long signal -1 , -1
#####

-1.0 -1.0
15445 #2 long signal -1 , -1
##########################################
1.0 -1.0
15461 #1 close signal +1, -1
1.0 1.0
15470 #2 close signal +1, +1
########################################
-1.0 1.0
15727 #1 long signal -1 , +1
-1.0 -1.0
15761 #2 long signal -1 , -1
##########################################
1.0 -1.0
15836 #1 close signal +1, -1
1.0 1.0
15845 #2 close signal +1, +1
########################################
-1.0 1.0
15870 #1 long signal -1 , +1
-1.0 -1.0
15898 #2 long signal -1 , -1
##########################################


In [99]:
renew_index_long = 0
holding_position = 0
# print(renew_index, '#0')

while True:
    for index, row in a.loc[renew_index_long:].iterrows():
    
    
    ######################################## Holding_Position == 0 ################################################
    
        if holding_position == 0:
            
#             print('Current Holding Position:', holding_position)
            
            ################################### 'Buy SIgnal Start' ##################################################
            
            
            if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] <= '34'): 
                renew_index_long = index
                print('Long Signal Found')
                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                 print(renew_index_long,'#1 long signal -1 , +1')
            
                for index, row in a.loc[renew_index_long:].iterrows():

                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
                        renew_index_long = index

                        holding_position += 1
                        
                        price_Long = a.loc[index,'현재가']
                        
                        print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Long 가격:', price_Long)
#                         print(renew_index_long, '#2 long signal -1 , -1')
                        print('Current Holding Position', holding_position)
#                         print('########################################')

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == 1 and row['Rest_Flag'] == -1:
                                renew_index_long = index
                                print('Long Closing Signal Found')
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                                 print(renew_index_long, '#1 Long_close signal +1, -1')

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1:
                                        renew_index_long = index
                                        
                                        holding_position -= 1
                                        
                                        price_Closing_Long = a.loc[index,'현재가']
                                        
                                        Long_p_and_l = price_Closing_Long - price_Long
                                
                                        print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index, 
                                              'Closing_Long_가격:',price_Closing_Long, 'Long Profit:', Long_p_and_l)
#                                         print(renew_index_long, '#2 Long_close signal +1, +1')
                                        print('Current Holding Position', holding_position)
                                        print('Long Closing Signal Finding is All Done')
                                        print('########################################')
                                        break
                                break
                    
                            elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                holding_position -= 1
                                price_Closing_Long_at_34 = a.loc[index, '현재가']
                                Long_p_and_l = price_Closing_Long_at_34 - price_Long
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                          'Closing_Long_at_34_가격:', price_Closing_Long_at_34, '34_Long Profit:', Long_p_and_l)
                                print('Current Holding Position', holding_position)
                                
                                break
                            
                        break
                break                    

            ################################### 'Short SIgnal Start' ##################################################       
                
            elif row['H1_Flag'] == 1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] <= '34'):               
                renew_index_long = index
                print('Short Signal Found')
                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                 print(renew_index_long,'#1 short signal +1 , -1')
            
                for index, row in a.loc[renew_index_long:].iterrows():

                    if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1:
                        
                        renew_index_long = index
                        holding_position += 1

                        price_Short = a.loc[index,'현재가']

                        print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Short 가격:', price_Short)
#                         print(renew_index_long, '#2 short signal +1 , +1')
                        print('Current Holding Position', holding_position)
#                         print('########################################')

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1:
                                renew_index_long = index
                                print('Short Closing Signal Found')
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                                 print(renew_index_long, '#1 Short_close signal -1, +1')

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
                                        renew_index_long = index
                                        
                                        holding_position -= 1
                                        
                                        price_Closing_Short = a.loc[index,'현재가']
                                        
                                        Short_p_and_l = price_Short - price_Closing_Short
                                
                                        print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index, 
                                              'Closing_Short_가격:',price_Closing_Short, 'Short Profit:', Short_p_and_l)
#                                         print(renew_index_long, '#2 Short_close signal -1, -1')
                                        print('Current Holding Position', holding_position)
                                        print('Short Closing Signal Finding is All Done')
                                        print('########################################')

                                        break
                                break
                            
                            elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                    holding_position -= 1
                                    price_Closing_Short_at_34 = a.loc[index, '현재가']
                                    Short_p_and_l = price_Short - price_Closing_Short_at_34
                                    print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                          'Closing_Short_at_34_가격:', price_Closing_Short_at_34, '34_Short Profit:', Short_p_and_l)
                                    print('Current Holding Position', holding_position)
                                    break

                        break
                break 
    if index == len(a.loc[:]):
        break                


Long Signal Found
-1.0 1.0 InDex: 227
-1.0 -1.0 InDex: 231 Long 가격: 10747.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 397
1.0 1.0 InDex: 408 Closing_Long_가격: 10746.0 Long Profit: -1.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 428
1.0 1.0 InDex: 444 Short 가격: 10747.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 543
-1.0 -1.0 InDex: 551 Closing_Short_가격: 10746.0 Short Profit: 1.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 553
-1.0 -1.0 InDex: 590 Long 가격: 10747.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 642
1.0 1.0 InDex: 678 Closing_Long_가격: 10746.0 Long Profit: -1.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 696

1.0 1.0 InDex: 6842 Closing_Long_가격: 10744.0 Long Profit: 0.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 6857
1.0 1.0 InDex: 6873 Short 가격: 10745.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 7559
-1.0 -1.0 InDex: 7572 Closing_Short_가격: 10747.0 Short Profit: -2.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 7616
-1.0 -1.0 InDex: 7640 Long 가격: 10747.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 7709
1.0 1.0 InDex: 7710 Closing_Long_가격: 10747.0 Long Profit: 0.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 7717
1.0 1.0 InDex: 7726 Short 가격: 10747.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 7860
-1.0 -1.0 InDex: 7869 Clos

Short Closing Signal Found
-1.0 1.0 InDex: 14522
-1.0 -1.0 InDex: 15445 Closing_Short_가격: 10750.0 Short Profit: 0.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 15461
1.0 1.0 InDex: 15470 Short 가격: 10750.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 15727
-1.0 -1.0 InDex: 15761 Closing_Short_가격: 10751.0 Short Profit: -1.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 15779
-1.0 -1.0 InDex: 15783 Long 가격: 10750.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 15836
1.0 1.0 InDex: 15845 Closing_Long_가격: 10751.0 Long Profit: 1.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 15870
-1.0 -1.0 InDex: 15898 Long 가격: 10751.0
Current Holding Position 1
-1.0 -1.0

In [102]:
a.loc[15929]

시간                   [15:34:39-646502]
현재가                              10752
매수1호가수량                            126
매도1호가수량                           1992
H1_Flag                             -1
Rest_Flag                           -1
long_signal                        NaN
long_close_signal                  NaN
Name: 15929, dtype: object

In [89]:
a.loc[15897, '시간'][1:3]

'15'

In [105]:
a.loc[15870]

시간                   [15:33:04-263606]
현재가                              10751
매수1호가수량                            NaN
매도1호가수량                            NaN
H1_Flag                             -1
Rest_Flag                            1
long_signal                        NaN
long_close_signal                  NaN
Name: 15870, dtype: object

In [104]:
renew_index_long = 0
holding_position = 0
# print(renew_index, '#0')

while True:
    for index, row in a.loc[renew_index_long:].iterrows():
    
    
    ######################################## Holding_Position == 0 ################################################
    
        if holding_position == 0:
            
#             print('Current Holding Position:', holding_position)
            
            ################################### 'Buy SIgnal Start' ##################################################
            
            
            if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'): 
                renew_index_long = index
                print('Long Signal Found')
                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                 print(renew_index_long,'#1 long signal -1 , +1')
            
                for index, row in a.loc[renew_index_long:].iterrows():

                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
                        renew_index_long = index

                        holding_position += 1
                        
                        price_Long = a.loc[index,'현재가']
                        
                        print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Long 가격:', price_Long)
#                         print(renew_index_long, '#2 long signal -1 , -1')
                        print('Current Holding Position', holding_position)
#                         print('########################################')

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == 1 and row['Rest_Flag'] == -1:
                                renew_index_long = index
                                print('Long Closing Signal Found')
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                                 print(renew_index_long, '#1 Long_close signal +1, -1')

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1:
                                        renew_index_long = index
                                        
                                        holding_position -= 1
                                        
                                        price_Closing_Long = a.loc[index,'현재가']
                                        
                                        Long_p_and_l = price_Closing_Long - price_Long
                                
                                        print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index, 
                                              'Closing_Long_가격:',price_Closing_Long, 'Long Profit:', Long_p_and_l)
#                                         print(renew_index_long, '#2 Long_close signal +1, +1')
                                        print('Current Holding Position', holding_position)
                                        print('Long Closing Signal Finding is All Done')
                                        print('########################################')
                                        break
                                break
                    
                            elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                holding_position -= 1
                                price_Closing_Long_at_34 = a.loc[index, '현재가']
                                Long_p_and_l = price_Closing_Long_at_34 - price_Long
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                          'Closing_Long_at_34_가격:', price_Closing_Long_at_34, '34_Long Profit:', Long_p_and_l)
                                print('Current Holding Position', holding_position)
                                
                                break
                            
                        break
                break                    

            ################################### 'Short SIgnal Start' ##################################################       
                
            elif row['H1_Flag'] == 1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):               
                renew_index_long = index
                print('Short Signal Found')
                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                 print(renew_index_long,'#1 short signal +1 , -1')
            
                for index, row in a.loc[renew_index_long:].iterrows():

                    if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1:
                        
                        renew_index_long = index
                        holding_position += 1

                        price_Short = a.loc[index,'현재가']

                        print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Short 가격:', price_Short)
#                         print(renew_index_long, '#2 short signal +1 , +1')
                        print('Current Holding Position', holding_position)
#                         print('########################################')

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1:
                                renew_index_long = index
                                print('Short Closing Signal Found')
                                print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)
#                                 print(renew_index_long, '#1 Short_close signal -1, +1')

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
                                        renew_index_long = index
                                        
                                        holding_position -= 1
                                        
                                        price_Closing_Short = a.loc[index,'현재가']
                                        
                                        Short_p_and_l = price_Short - price_Closing_Short
                                
                                        print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index, 
                                              'Closing_Short_가격:',price_Closing_Short, 'Short Profit:', Short_p_and_l)
#                                         print(renew_index_long, '#2 Short_close signal -1, -1')
                                        print('Current Holding Position', holding_position)
                                        print('Short Closing Signal Finding is All Done')
                                        print('########################################')

                                        break
                                break
                            
                            elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                    holding_position -= 1
                                    price_Closing_Short_at_34 = a.loc[index, '현재가']
                                    Short_p_and_l = price_Short - price_Closing_Short_at_34
                                    print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                          'Closing_Short_at_34_가격:', price_Closing_Short_at_34, '34_Short Profit:', Short_p_and_l)
                                    print('Current Holding Position', holding_position)
                                    break

                        break
                break 
    if index == len(a.loc[:]):
        break                


Long Signal Found
-1.0 1.0 InDex: 227
-1.0 -1.0 InDex: 231 Long 가격: 10747.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 397
1.0 1.0 InDex: 408 Closing_Long_가격: 10746.0 Long Profit: -1.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 428
1.0 1.0 InDex: 444 Short 가격: 10747.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 543
-1.0 -1.0 InDex: 551 Closing_Short_가격: 10746.0 Short Profit: 1.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 553
-1.0 -1.0 InDex: 590 Long 가격: 10747.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 642
1.0 1.0 InDex: 678 Closing_Long_가격: 10746.0 Long Profit: -1.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 696

-1.0 -1.0 InDex: 10300 Closing_Short_가격: 10748.0 Short Profit: 1.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 11598
1.0 1.0 InDex: 12001 Short 가격: 10752.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 12124
-1.0 -1.0 InDex: 12132 Closing_Short_가격: 10752.0 Short Profit: 0.0
Current Holding Position 0
Short Closing Signal Finding is All Done
########################################
Long Signal Found
-1.0 1.0 InDex: 12135
-1.0 -1.0 InDex: 12137 Long 가격: 10752.0
Current Holding Position 1
Long Closing Signal Found
1.0 -1.0 InDex: 12219
1.0 1.0 InDex: 12221 Closing_Long_가격: 10752.0 Long Profit: 0.0
Current Holding Position 0
Long Closing Signal Finding is All Done
########################################
Short Signal Found
1.0 -1.0 InDex: 12255
1.0 1.0 InDex: 12264 Short 가격: 10752.0
Current Holding Position 1
Short Closing Signal Found
-1.0 1.0 InDex: 12743
-1.0 -1.0 

In [107]:
a.loc[15870], a.loc[15898]

(시간                   [15:33:04-263606]
 현재가                              10751
 매수1호가수량                            NaN
 매도1호가수량                            NaN
 H1_Flag                             -1
 Rest_Flag                            1
 long_signal                        NaN
 long_close_signal                  NaN
 Name: 15870, dtype: object, 시간                   [15:34:04-029952]
 현재가                              10751
 매수1호가수량                             21
 매도1호가수량                           1587
 H1_Flag                             -1
 Rest_Flag                           -1
 long_signal                        NaN
 long_close_signal                  NaN
 Name: 15898, dtype: object)

In [114]:
a.loc[15870, '시간'][1:3]

'15'

In [120]:
a.loc[15870, '시간'][1:3] <= '14'

False